In [ ]:
!pip install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=585b3fd36a15bc543977afab9be7457a1e7386faeaa7bd4ac05ca7926b31ae5c
  Stored in directory: /root/.cache/pip/wheels/f0/3f/76/8783033e8524d407e1bebaf72fdd3f3eba27e0c030e92bbd87
Successfully built chess


In [ ]:
import numpy as np
import chess
import chess.svg

# Piece values for heuristic evaluation
PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000,
}

# Most Valuable Victim - Least Valuable Attacker (MVV-LVA) move ordering
def mvv_lva(move, board):
    if board.is_capture(move):
        victim = board.piece_at(move.to_square)
        attacker = board.piece_at(move.from_square)
        return (PIECE_VALUES.get(victim.piece_type, 0) - PIECE_VALUES.get(attacker.piece_type, 0))
    return 0

# Heuristic Evaluation Function
def evaluate_board(board):
    if board.is_checkmate():
        return -9999 if board.turn else 9999
    if board.is_stalemate() or board.is_insufficient_material():
        return 0

    score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            value = PIECE_VALUES.get(piece.piece_type, 0)
            score += value if piece.color == chess.WHITE else -value
    return score

# Minimax with Alpha-Beta Pruning
def minimax(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board), None

    best_move = None
    legal_moves = sorted(board.legal_moves, key=lambda move: mvv_lva(move, board), reverse=True)

    if maximizing_player:
        max_eval = -np.inf
        for move in legal_moves:
            board.push(move)
            eval, _ = minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = np.inf
        for move in legal_moves:
            board.push(move)
            eval, _ = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

# Chess Bot Class
class ChessBot:
    def __init__(self, depth=3):
        self.board = chess.Board()
        self.depth = depth

    def play_best_move(self):
        _, best_move = minimax(self.board, self.depth, -np.inf, np.inf, self.board.turn)
        if best_move:
            self.board.push(best_move)

    def play_game(self):
        while not self.board.is_game_over():
            print(self.board)
            self.play_best_move()
        print("Game Over! Result:", self.board.result())

if __name__ == "__main__":
    bot = ChessBot(depth=3)
    bot.play_game()

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . N .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R
r n b q k b . r
p p p p p p p p
. . . . . . . .
. . . . . . N .
. . . . . . n .
. . . . . . . .
P P P P P P P P
R N B Q K B . R
r n b q k b . r
p p p p p p p p
. . . . . . . .
. . . . . . N .
. . . . . . n .
. . N . . . . .
P P P P P P P P
R . B Q K B . R
r . b q k b . r
p p p p p p p p
. . n . . . . .
. . . . . . N .
. . . . . . n .
. . N . . . . .
P P P P P P P P
R . B Q K B . R
r . b q k b . r
p p p p p p p p
. . n . . . . .
. . . N . . N .
. . . . . . n .
. . . . . . . .
P P P P 